<a href="https://colab.research.google.com/github/SG60/2021-multi-group-piece/blob/main/Tempos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Setup

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

from sympy import init_session
init_session()

IPython console for SymPy 1.8 (Python 3.9.2-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.8/



In [2]:
# Tool for custom rounding
from bisect import bisect_left

class CustomRound:

    def __init__(self,iterable):
        self.data = sorted(iterable)

    def __call__(self,x):
        data = self.data
        ndata = len(data)
        idx = bisect_left(data,x)
        if idx <= 0:
            return data[0]
        elif idx >= ndata:
            return data[ndata-1]
        x0 = data[idx-1]
        x1 = data[idx]
        if abs(x-x0) < abs(x-x1):
            return x0
        return x1

custom_round_values_base = [0, 1/5,1/4,1/3,2/5,1/2,3/5,2/3,3/4,4/5]
custom_round_values = [i+j for i in range(20) for j in custom_round_values_base]
custom_round = CustomRound(custom_round_values)

# assigments

In [3]:
a, a1, b, b1, c, c1, s, t = symbols('a:2 b:2 c:2 s t')

# Ramp code

In [4]:
def ramp(start_tempo,end_tempo,total_beats,bound_beat):
  '''This integral is equal to the timecode at the bound_beat.'''
  return integrate(1/(start_tempo/60+(x/total_beats)*(S(end_tempo)/60-start_tempo/60)),(x,0,bound_beat))

In [5]:
tempoeq2_version_using_ramp = Eq(ramp(a,b,c,20) + ramp(b,b1,c1,s), t)
display(tempoeq2_version_using_ramp) # this is equal to tempoeq2
# or another way, starting with an unknown function:
test1 = Eq(f(a,b,c,20)+f(b,b1,c1,s),t)
display(test1)
display(test1.replace(f,ramp))

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + 20⋅a₀ - 20⋅b₀)   60⋅c₁⋅log(-b₀⋅c₁)   60
───────────────── - ───────────────────────────────── + ───────────────── - ──
     a₀ - b₀                     a₀ - b₀                     b₀ - b₁          

⋅c₁⋅log(-b₀⋅c₁ + s⋅(b₀ - b₁))    
───────────────────────────── = t
          b₀ - b₁                

f(a₀, b₀, c₀, 20) + f(b₀, b₁, c₁, s) = t

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + 20⋅a₀ - 20⋅b₀)   60⋅c₁⋅log(-b₀⋅c₁)   60
───────────────── - ───────────────────────────────── + ───────────────── - ──
     a₀ - b₀                     a₀ - b₀                     b₀ - b₁          

⋅c₁⋅log(-b₀⋅c₁ + s⋅(b₀ - b₁))    
───────────────────────────── = t
          b₀ - b₁                

#Wind up+down ramp in A section

##Up Ramp

In [6]:
a, a1, b, b1, c, c1, s, t = symbols('a:2 b:2 c:2 s t')
# a = start tempo of ramp
# b = end tempo of ramp
# c = length of ramp in beats
# s = beat we are interested in
# t = timecode of that beat
tempoeq = Eq(integrate(1/(a/60+(x/c)*(b/60-a/60)),(x,0,s)),t)
tempoeq

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + s⋅(a₀ - b₀))    
───────────────── - ─────────────────────────────── = t
     a₀ - b₀                    a₀ - b₀                

In [7]:
Eq(Integral(1/(a/60+(x/c)*(b/60-a/60)),(x,0,s)),t)

s                          
⌠                          
⎮         1                
⎮ ────────────────── dx = t
⎮        ⎛  a₀   b₀⎞       
⎮      x⋅⎜- ── + ──⎟       
⎮ a₀     ⎝  60   60⎠       
⎮ ── + ─────────────       
⎮ 60         c₀            
⌡                          
0                          

In [8]:
s_solved = solveset(tempoeq,s).args[0]
display(s_solved)

                   ⎛60⋅c₀⋅log(-a₀⋅c₀)    ⎞
         (a₀ - b₀)⋅⎜───────────────── - t⎟
                   ⎝     a₀ - b₀         ⎠
         ─────────────────────────────────
                       60⋅c₀              
a₀⋅c₀ + ℯ                                 
──────────────────────────────────────────
                 a₀ - b₀                  

In [9]:
#@title Values for steady part
number_of_beats = 13 #@param {type:"integer"}
steady_tempo = 96 #@param {type:"integer"}

In [10]:
#@title Values for ramp part
start_tempo = 96 #@param {type:"integer"}
end_tempo = 240 #@param {type:"integer"}
length_in_beats = 20 #@param {type:"integer"}

In [11]:
temposteady = steady_tempo
beatlength = S(60)/steady_tempo
#beats = [0, S(1)/3, 2, 4]
beats = range(number_of_beats)
beattimecodes_up_ramp = [beatlength*i for i in beats]
outbeats = [s_solved.n(6,{a: start_tempo, b: end_tempo, c: length_in_beats, t: i},chop=True) for i in beattimecodes_up_ramp]
for i in range(len(outbeats)):
  print(beats[i], outbeats[i])

0 0
1 1.03846
2 2.15779
3 3.36430
4 4.66478
5 6.06655
6 7.57750
7 9.20612
8 10.9616
9 12.8538
10 14.8933
11 17.0917
12 19.4614


In [12]:
print(1/5,1/4,1/3,2/5,1/2,3/5,2/3,3/4,4/5)

0.2 0.25 0.3333333333333333 0.4 0.5 0.6 0.6666666666666666 0.75 0.8


In [13]:
display(beattimecodes_up_ramp[12].n())
display([i.n() for i in beattimecodes_up_ramp])

7.50000000000000

[0, 0.625, 1.25, 1.875, 2.5, 3.125, 3.75, 4.375, 5.0, 5.625, 6.25, 6.875, 7.5]

In [14]:
# Timecode at beat 20
solveset(tempoeq.subs({a: start_tempo, b: end_tempo, c: length_in_beats, s:20}),t).n(6)

{7.63576}

##Down Ramp

In [15]:
# b1 = end tempo of down ramp
# c1 = number of beats in down ramp
tempoeq2_not_done = Eq(Integral(1/(a/60+(x/c)*(b/60-a/60)),(x,0,20))+Integral(1/(b/60+(x/c1)*(b1/60-b/60)),(x,0,s)),t)
display(tempoeq2_not_done)
tempoeq2 = tempoeq2_not_done.doit()
display(tempoeq2)

20                         s                          
⌠                          ⌠                          
⎮          1               ⎮         1                
⎮  ────────────────── dx + ⎮ ────────────────── dx = t
⎮         ⎛  a₀   b₀⎞      ⎮        ⎛  b₀   b₁⎞       
⎮       x⋅⎜- ── + ──⎟      ⎮      x⋅⎜- ── + ──⎟       
⎮  a₀     ⎝  60   60⎠      ⎮ b₀     ⎝  60   60⎠       
⎮  ── + ─────────────      ⎮ ── + ─────────────       
⎮  60         c₀           ⎮ 60         c₁            
⌡                          ⌡                          
0                          0                          

60⋅c₀⋅log(-a₀⋅c₀)   60⋅c₀⋅log(-a₀⋅c₀ + 20⋅a₀ - 20⋅b₀)   60⋅c₁⋅log(-b₀⋅c₁)   60
───────────────── - ───────────────────────────────── + ───────────────── - ──
     a₀ - b₀                     a₀ - b₀                     b₀ - b₁          

⋅c₁⋅log(-b₀⋅c₁ + s⋅(b₀ - b₁))    
───────────────────────────── = t
          b₀ - b₁                

In [16]:
s2_solved = solveset(tempoeq2,s).args[0]
#display(s2_solved.args[0])
s2_solved_subbed = s2_solved.subs({a: 96, b: 240, c: 20, b1: 60, c1: 16})
display(s2_solved_subbed)
# values for slowing down ramp
beats_down_ramp = range(13,floor(16/beatlength))
beattimecodes_down_ramp = [beatlength*i for i in beats_down_ramp]
display([i.n() for i in beattimecodes_down_ramp])
outbeats_down_ramp = [s2_solved_subbed.n(6,{t: i},chop=True) for i in beattimecodes_down_ramp]
for i in range(len(outbeats_down_ramp)):
  print(beats_down_ramp[i],
        outbeats_down_ramp[i],
        custom_round(outbeats_down_ramp[i])
        ) # beat values (from start of ramp)

                     -3⋅t 
                     ─────
         7/16  9/16    16 
64   80⋅2    ⋅5    ⋅ℯ     
── - ─────────────────────
3              3          

[8.125, 8.75, 9.375, 10.0, 10.625, 11.25, 11.875, 12.5, 13.125, 13.75, 14.375,
 15.0]

13 1.86990 1.8
14 4.02220 4
15 5.93649 6
16 7.63910 7.666666666666667
17 9.15343 9.2
18 10.5003 10.5
19 11.6982 11.666666666666666
20 12.7637 12.75
21 13.7113 13.75
22 14.5542 14.6
23 15.3038 15.333333333333334
24 15.9706 16


In [17]:
#total length of both ramps in seconds:
solveset(tempoeq2,t).args[0].subs({a: 96, b: 240, c: 20, b1: 60, c1: 16, s:16}).n(6)

15.0293

# New section

In [18]:
# String accel section length in seconds
ramp(60,180,9*4,9*4)*2

39.5500423920519

# Tempo relations in piece

In [19]:
# tempo
tempolist = [128/0.8, 128]
tempolist.append(tempolist[-1]*(3/4))
tempolist.extend([240,60])
tempolist.append(128)
tempolist.append(180)
tempolist.append(tempolist[-1]*0.5)
display(tempolist)

[160.0, 128, 96.0, 240, 60, 128, 180, 90.0]

# Middle Section

In [20]:
# downwards ramp
middle_ramp_1_total_length = (5+4+4+5+4+3+4)*4
middle_ramp_1_f = Eq(f(180,90,middle_ramp_1_total_length,s),t)
display(middle_ramp_1_f)
middle_ramp_1 = middle_ramp_1_f.replace(f,ramp)
print("length of one downwards ramp in seconds:")
display(middle_ramp_1.subs({s: middle_ramp_1_total_length}).n(3))

f(180, 90, 116, s) = t

length of one downwards ramp in seconds:


53.6 = t